In [1]:
import skrub
import pandas as pd
from sklearn.preprocessing import OrdinalEncoder, OneHotEncoder

In [2]:
df = pd.read_csv('../cat-in-the-dat-ii/input/train.csv')

In [3]:
data = skrub.var("data", df)
X = data.drop(["target","id"], axis=1).skb.mark_as_X()
y = data["target"].skb.mark_as_y()

In [4]:
bin_ord_selector = skrub.selectors.filter_names(lambda name: "bin" in name or "ord" in name)
low_cardinality_nom_selector = skrub.selectors.filter(lambda col: "nom" in col.name and col.nunique() < 10)
high_cardinality_nom_selector = skrub.selectors.filter(lambda col: "nom" in col.name and col.nunique() >= 10)

In [5]:
ordinal_encoder = OrdinalEncoder()
one_hot_encoder = OneHotEncoder(handle_unknown="ignore", sparse_output=False)

In [6]:
vectorizer = skrub.TableVectorizer(
    specific_transformers=[
        (ordinal_encoder, bin_ord_selector),
        (one_hot_encoder, low_cardinality_nom_selector),
        ("passthrough", high_cardinality_nom_selector)
    ],
)

In [10]:
vec_X = X.skb.apply(vectorizer)
actual_vec_X = vec_X.skb.preview()

In [11]:
bin_ord_cols = X.skb.apply_func(bin_ord_selector.expand)
low_cardinality_nom_cols = X.skb.apply_func(low_cardinality_nom_selector.expand)
high_cardinality_nom_cols = X.skb.apply_func(high_cardinality_nom_selector.expand)

In [12]:
vectorizer2 = skrub.TableVectorizer(
    specific_transformers=[
        (ordinal_encoder, bin_ord_cols),
        (one_hot_encoder, low_cardinality_nom_cols),
        ("passthrough", high_cardinality_nom_cols)
    ],
)

In [13]:
vec_X2 = X.skb.apply(vectorizer2)
actual_vec_X2 = vec_X2.skb.preview()

In [15]:
type(actual_vec_X2), type(actual_vec_X)

(pandas.core.frame.DataFrame, pandas.core.frame.DataFrame)

In [18]:
actual_vec_X.head()

,bin_0,bin_1,bin_2,bin_3,bin_4,nom_0_Blue,nom_0_Green,nom_0_Red,nom_0_nan,nom_1_Circle,...,nom_8,nom_9,ord_0,ord_1,ord_2,ord_3,ord_4,ord_5,day,month
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0256c7a4b,02e7c8990,2.0,0.0,3.0,2.0,20.0,56.0,6.0,3.0
1,1.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,...,52ead350c,f37df64af,2.0,2.0,5.0,4.0,23.0,150.0,7.0,7.0
2,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,745b909d1,NaN,2.0,NaN,2.0,13.0,15.0,105.0,5.0,9.0
3,NaN,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,...,bdaa56dd1,f9d456e57,0.0,4.0,4.0,0.0,2.0,NaN,3.0,3.0
4,0.0,NaN,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,...,NaN,c5361037c,2.0,2.0,1.0,7.0,2.0,50.0,5.0,12.0


In [46]:
vec_X2 = X.skb.apply(vectorizer2)

RuntimeError: Evaluation of 'apply()' failed.
You can see the full traceback above. The error message was:
ValueError: 'specific_transformers' must be a list of (transformer, list of columns) pairs. Got (OrdinalEncoder(), filter_names(<lambda>)) at index 0.